# Generación de Datos sintéticos de entrenamiento para el nano language model

El nano language model, desde ahora en adelante NLM, es un modelo de lenguaje especializado en transformar lenguaje natural en formulas de lógica proposicional. Para entrenar este modelo, es necesario contar con un conjunto de datos de entrenamiento que contenga ejemplos de lenguaje natural y sus correspondientes formulas de lógica proposicional. Para esto le pediremos a un LLM más grande, como GPT-4 o DeepSeek R1 que nos genere nuestro dataset de entrenamiento sintético.

Para esto usaremos Pydantic AI para usar la api de DeepSeek R1 usando la modalidad de JSON Output, lo que nos permitirá generar un dataset de entrenamiento estructurado y fácil de usar para entrenar nuestro NLM.

La estructura de nuestro dataset de entrenamiento será la siguiente:

In [1]:
from pydantic import BaseModel, Field
from enum import Enum
from IPython.display import Markdown
import json


# --- Enums para valores controlados ---


class LogicalConnector(str, Enum):
    """Conectores lógicos estándar de la lógica proposicional."""
    NEGATION = "¬"  # NOT
    CONJUNCTION = "∧"  # AND
    DISJUNCTION = "∨"  # OR
    IMPLICATION = "→"  # IF...THEN
    BICONDITIONAL = "↔"


class Complexity(str, Enum):
    """Nivel de complejidad de la proposición."""

    SIMPLE = "simple"  # 1-2 átomos, 1 conector
    INTERMEDIATE = "intermediate"  # 2-3 átomos, 2-3 conectores
    ADVANCED = "advanced"  # 3+ átomos, conectores anidados


# --- Sub-modelos ---


class ReasoningStep(BaseModel):
    """Un paso individual en el proceso de razonamiento."""

    step: int = Field(description="Número secuencial del paso de razonamiento.")
    explanation: str = Field(
        description="Descripción técnica de lo que se está analizando en este punto del razonamiento."
    )


class AtomDefinition(BaseModel):
    """Mapeo entre una letra proposicional y su significado en lenguaje natural."""

    atom: str = Field(
        description="La letra que representa la proposición (ejemplo: p, q, r...).",
        pattern=r"^[a-z]$",
    )
    definition: str = Field(
        description="La proposición simple en lenguaje natural que representa el átomo."
    )


class ConnectorUsage(BaseModel):
    """Registro de un conector lógico usado y la frase que lo activó."""

    connector: LogicalConnector = Field(description="El conector lógico identificado.")
    natural_language_cue: str = Field(
        description="La palabra o frase en el texto original que indica este conector (ejemplo: 'y', 'si...entonces', 'o', 'no es cierto que')."
    )


# --- Bloque de pensamiento (chain-of-thought) ---


class ThoughtBlock(BaseModel):
    """Bloque de razonamiento completo: análisis paso a paso del texto."""

    reasoning_steps: list[ReasoningStep] = Field(
        description="Secuencia de pasos de razonamiento para descomponer el texto en lógica proposicional."
    )
    identified_atoms: list[AtomDefinition] = Field(
        description="Lista de átomos proposicionales identificados en el texto."
    )
    identified_connectors: list[ConnectorUsage] = Field(
        description="Lista de conectores lógicos identificados junto con su pista en el texto original."
    )


# --- Resultado final ---


class PropositionalFormula(BaseModel):
    """La fórmula final en lógica proposicional."""

    formula: str = Field(
        description="La fórmula en lógica proposicional usando símbolos estándar (ejemplo: (p ∧ q) → r)."
    )
    formula_ascii: str = Field(
        description="La misma fórmula usando notación ASCII (ejemplo: (p & q) -> r)."
    )


# --- Ejemplo de entrenamiento completo ---


class TrainingExample(BaseModel):
    """Un ejemplo completo de entrenamiento: texto natural → lógica proposicional."""

    natural_language_input: str = Field(
        description="El enunciado o argumento en lenguaje natural que se debe formalizar."
    )
    complexity: Complexity = Field(description="Nivel de complejidad del ejemplo.")
    thought: ThoughtBlock = Field(
        description="Bloque de pensamiento con el razonamiento paso a paso (chain-of-thought)."
    )
    output: PropositionalFormula = Field(
        description="La fórmula proposicional resultante."
    )


# --- Dataset completo ---


class SyntheticDataset(BaseModel):
    """Dataset sintético de entrenamiento para el NLM."""

    examples: list[TrainingExample] = Field(
        description="Lista de ejemplos de entrenamiento para transformar lenguaje natural en lógica proposicional."
    )


# Visualizar el JSON Schema con syntax highlighting
schema_json = json.dumps(SyntheticDataset.model_json_schema(), indent=2, ensure_ascii=False)
Markdown(f"```json\n{schema_json}\n```")

```json
{
  "$defs": {
    "AtomDefinition": {
      "description": "Mapeo entre una letra proposicional y su significado en lenguaje natural.",
      "properties": {
        "atom": {
          "description": "La letra que representa la proposición (ejemplo: p, q, r...).",
          "pattern": "^[a-z]$",
          "title": "Atom",
          "type": "string"
        },
        "definition": {
          "description": "La proposición simple en lenguaje natural que representa el átomo.",
          "title": "Definition",
          "type": "string"
        }
      },
      "required": [
        "atom",
        "definition"
      ],
      "title": "AtomDefinition",
      "type": "object"
    },
    "Complexity": {
      "description": "Nivel de complejidad de la proposición.",
      "enum": [
        "simple",
        "intermediate",
        "advanced"
      ],
      "title": "Complexity",
      "type": "string"
    },
    "ConnectorUsage": {
      "description": "Registro de un conector lógico usado y la frase que lo activó.",
      "properties": {
        "connector": {
          "$ref": "#/$defs/LogicalConnector",
          "description": "El conector lógico identificado."
        },
        "natural_language_cue": {
          "description": "La palabra o frase en el texto original que indica este conector (ejemplo: 'y', 'si...entonces', 'o', 'no es cierto que').",
          "title": "Natural Language Cue",
          "type": "string"
        }
      },
      "required": [
        "connector",
        "natural_language_cue"
      ],
      "title": "ConnectorUsage",
      "type": "object"
    },
    "LogicalConnector": {
      "description": "Conectores lógicos estándar de la lógica proposicional.",
      "enum": [
        "¬",
        "∧",
        "∨",
        "→",
        "↔"
      ],
      "title": "LogicalConnector",
      "type": "string"
    },
    "PropositionalFormula": {
      "description": "La fórmula final en lógica proposicional.",
      "properties": {
        "formula": {
          "description": "La fórmula en lógica proposicional usando símbolos estándar (ejemplo: (p ∧ q) → r).",
          "title": "Formula",
          "type": "string"
        },
        "formula_ascii": {
          "description": "La misma fórmula usando notación ASCII (ejemplo: (p & q) -> r).",
          "title": "Formula Ascii",
          "type": "string"
        }
      },
      "required": [
        "formula",
        "formula_ascii"
      ],
      "title": "PropositionalFormula",
      "type": "object"
    },
    "ReasoningStep": {
      "description": "Un paso individual en el proceso de razonamiento.",
      "properties": {
        "step": {
          "description": "Número secuencial del paso de razonamiento.",
          "title": "Step",
          "type": "integer"
        },
        "explanation": {
          "description": "Descripción técnica de lo que se está analizando en este punto del razonamiento.",
          "title": "Explanation",
          "type": "string"
        }
      },
      "required": [
        "step",
        "explanation"
      ],
      "title": "ReasoningStep",
      "type": "object"
    },
    "ThoughtBlock": {
      "description": "Bloque de razonamiento completo: análisis paso a paso del texto.",
      "properties": {
        "reasoning_steps": {
          "description": "Secuencia de pasos de razonamiento para descomponer el texto en lógica proposicional.",
          "items": {
            "$ref": "#/$defs/ReasoningStep"
          },
          "title": "Reasoning Steps",
          "type": "array"
        },
        "identified_atoms": {
          "description": "Lista de átomos proposicionales identificados en el texto.",
          "items": {
            "$ref": "#/$defs/AtomDefinition"
          },
          "title": "Identified Atoms",
          "type": "array"
        },
        "identified_connectors": {
          "description": "Lista de conectores lógicos identificados junto con su pista en el texto original.",
          "items": {
            "$ref": "#/$defs/ConnectorUsage"
          },
          "title": "Identified Connectors",
          "type": "array"
        }
      },
      "required": [
        "reasoning_steps",
        "identified_atoms",
        "identified_connectors"
      ],
      "title": "ThoughtBlock",
      "type": "object"
    },
    "TrainingExample": {
      "description": "Un ejemplo completo de entrenamiento: texto natural → lógica proposicional.",
      "properties": {
        "natural_language_input": {
          "description": "El enunciado o argumento en lenguaje natural que se debe formalizar.",
          "title": "Natural Language Input",
          "type": "string"
        },
        "complexity": {
          "$ref": "#/$defs/Complexity",
          "description": "Nivel de complejidad del ejemplo."
        },
        "thought": {
          "$ref": "#/$defs/ThoughtBlock",
          "description": "Bloque de pensamiento con el razonamiento paso a paso (chain-of-thought)."
        },
        "output": {
          "$ref": "#/$defs/PropositionalFormula",
          "description": "La fórmula proposicional resultante."
        }
      },
      "required": [
        "natural_language_input",
        "complexity",
        "thought",
        "output"
      ],
      "title": "TrainingExample",
      "type": "object"
    }
  },
  "description": "Dataset sintético de entrenamiento para el NLM.",
  "properties": {
    "examples": {
      "description": "Lista de ejemplos de entrenamiento para transformar lenguaje natural en lógica proposicional.",
      "items": {
        "$ref": "#/$defs/TrainingExample"
      },
      "title": "Examples",
      "type": "array"
    }
  },
  "required": [
    "examples"
  ],
  "title": "SyntheticDataset",
  "type": "object"
}
```

Con el JSON Schema anterior, definimos la estructura de nuestro dataset de entrenamiento, que consiste en una lista de ejemplos, donde cada ejemplo contiene un enunciado en lenguaje natural y su correspondiente formula de lógica proposicional.

## Configuración del Agente de Generación

Configuramos el agente de Pydantic AI con DeepSeek como proveedor. Cargamos la API key desde el archivo `.env` y definimos el system prompt que guiará al modelo en la generación de ejemplos de entrenamiento.

In [2]:
import os
from dotenv import load_dotenv
from pydantic_ai import Agent

# Cargar variables de entorno desde .env
load_dotenv()

# Verificar que la API key está configurada
assert os.getenv("DEEPSEEK_API_KEY"), "DEEPSEEK_API_KEY no encontrada en .env"
print("API Key cargada correctamente ✓")

# System prompt especializado en dev/hacking/cybersec
SYSTEM_PROMPT = """Eres un experto en lógica proposicional especializado en ciberseguridad, desarrollo de software y hacking ético.

Tu tarea es generar ejemplos de entrenamiento que transformen enunciados técnicos en lenguaje natural
a fórmulas de lógica proposicional.

DOMINIOS TEMÁTICOS (varía entre estos):
- 🔓 Ciberseguridad: reglas de firewall, detección de intrusos, análisis de vulnerabilidades, políticas de acceso
- 🐛 Pentesting/CTF: condiciones de exploit, escalación de privilegios, movimiento lateral, exfiltración
- 💻 Programación: validaciones, flujos de control, condiciones de error, lógica de negocio
- 🖥️ Sysadmin: reglas de red, permisos Unix, configuración de servicios, monitoreo
- 🚀 DevOps/CI-CD: pipelines, condiciones de deploy, rollbacks, health checks
- 🎮 Game hacking: manipulación de memoria, bypass de anticheat, condiciones de win/lose

REGLAS:
1. Los enunciados deben sonar como los diría un dev/hacker real, con jerga técnica natural.
   Ejemplo: "Si el puerto 443 está abierto y el certificado SSL ha expirado, entonces el servidor es vulnerable a MITM"
2. Usa correctamente los conectores lógicos:
   - "y", "además", "siempre que ambos" → ∧ (conjunción)
   - "o", "ya sea", "cualquiera de" → ∨ (disyunción)
   - "si...entonces", "implica", "cuando", "siempre que" → → (implicación)
   - "si y solo si", "equivale a", "únicamente cuando" → ↔ (bicondicional)
   - "no", "no es cierto que", "falla", "no está" → ¬ (negación)
3. Los átomos deben ser letras minúsculas (p, q, r, s, t...).
4. Las fórmulas deben usar paréntesis para desambiguar precedencia.
5. Genera una mezcla de complejidades: simple, intermediate y advanced.
6. El razonamiento (thought) debe ser detallado paso a paso, explicando la lógica técnica.
7. Proporciona tanto la fórmula con símbolos Unicode (∧, ∨, →, ↔, ¬) como en ASCII (&, |, ->, <->, ~).
8. Genera los enunciados en español, pero permite términos técnicos en inglés cuando sea natural
   (ej: "firewall", "buffer overflow", "SQL injection", "deploy", "rollback").
9. NO generes enunciados genéricos aburridos. Cada ejemplo debe sentirse como algo que un profesional diría en su día a día.
"""

# Crear el agente con DeepSeek
agent = Agent(
    "deepseek:deepseek-chat",
    output_type=SyntheticDataset,
    system_prompt=SYSTEM_PROMPT,
)

print("Agente configurado correctamente ✓")

API Key cargada correctamente ✓
Agente configurado correctamente ✓


## Generación del Dataset

Generamos el dataset en lotes (batches) para evitar timeouts y poder ir guardando progreso. Cada lote le pide al modelo 5 ejemplos con una mezcla de complejidades.

In [3]:
import random
import pathlib


# --- Configuración de generación ---
TOTAL_EXAMPLES = 50
EXAMPLES_PER_BATCH = 1     # Reducido para evitar OOM
OUTPUT_FILE = "dataset.json"

# Temas especializados en dev/hacking/cybersec
TOPICS = [
    "reglas de firewall y filtrado de paquetes (iptables, WAF, ACLs)",
    "pentesting y explotación de vulnerabilidades (SQLi, XSS, RCE, SSRF)",
    "escalación de privilegios en Linux (SUID, capabilities, kernel exploits)",
    "CTF challenges (crypto, reversing, pwn, web)",
    "validaciones y sanitización de input en APIs REST",
    "flujos de autenticación y autorización (OAuth, JWT, RBAC)",
    "configuración de redes y segmentación (VLANs, subnets, VPN)",
    "pipelines CI/CD y condiciones de deploy (GitHub Actions, Jenkins)",
    "monitoreo y alertas de seguridad (SIEM, IDS/IPS, logs)",
    "game hacking y anti-cheat (memory manipulation, packet tampering)",
    "hardening de servidores y buenas prácticas sysadmin",
    "análisis de malware y condiciones de ejecución de payloads",
    "lógica de negocio en aplicaciones web (e-commerce, banking)",
    "permisos Unix y control de acceso (chmod, chown, sudo, SELinux)",
    "condiciones de error handling y excepciones en código",
]

all_examples: list[TrainingExample] = []

# Cargar progreso previo si existe
if pathlib.Path(OUTPUT_FILE).exists():
    with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
        prev = SyntheticDataset.model_validate_json(f.read())
        all_examples = prev.examples
        print(f"📂 Cargados {len(all_examples)} ejemplos previos")

async def generate_batch(topic: str, complexity_mix: str) -> list[TrainingExample]:
    """Genera un lote de ejemplos de entrenamiento."""
    prompt = f"""Genera exactamente {EXAMPLES_PER_BATCH} ejemplos de entrenamiento sobre: {topic}.

Mezcla de complejidades para este lote: {complexity_mix}.

Cada ejemplo debe tener:
- Un enunciado que suene como algo que diría un dev o hacker en su día a día
- Razonamiento detallado paso a paso con contexto técnico
- Identificación correcta de átomos y conectores
- La fórmula proposicional correcta en Unicode y ASCII

¡Sé creativo y técnicamente preciso! Usa jerga real del campo."""

    result = await agent.run(prompt)
    return result.output.examples

def save_progress():
    """Guarda el progreso actual a disco."""
    dataset = SyntheticDataset(examples=all_examples)
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        f.write(dataset.model_dump_json(indent=2))

async def generate_dataset():
    """Genera el dataset completo en lotes con guardado incremental."""
    remaining = TOTAL_EXAMPLES - len(all_examples)
    num_batches = remaining // EXAMPLES_PER_BATCH

    if num_batches == 0:
        print("✅ Ya se alcanzó el total de ejemplos deseado")
        return

    for i in range(num_batches):
        topic = random.choice(TOPICS)
        complexity_mixes = [
            "2 simple, 1 intermediate",
            "1 simple, 2 intermediate",
            "1 intermediate, 2 advanced",
            "1 simple, 1 intermediate, 1 advanced",
            "2 intermediate, 1 advanced",
        ]
        mix = complexity_mixes[i % len(complexity_mixes)]

        print(f"🔄 Lote {i+1}/{num_batches} | Tema: {topic[:50]}... | Mix: {mix}")

        try:
            examples = await generate_batch(topic, mix)
            all_examples.extend(examples)
            save_progress()  # Guardar después de cada lote
            print(f"   ✅ +{len(examples)} ejemplos (total: {len(all_examples)}) [guardado]")
        except Exception as e:
            print(f"   ❌ Error en lote {i+1}: {e}")
            continue

    print(f"\n🎉 Generación completada: {len(all_examples)} ejemplos en total")

await generate_dataset()

: 

## Guardar Dataset

Guardamos el dataset generado en formato JSON para usarlo en el entrenamiento del NLM.

In [ ]:
from collections import Counter

# Guardar el dataset en JSON
dataset = SyntheticDataset(examples=all_examples)
dataset_json = dataset.model_dump_json(indent=2)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(dataset_json)

print(f"✅ Dataset guardado en {OUTPUT_FILE}")
print(f"📊 Total de ejemplos: {len(all_examples)}")

# Mostrar estadísticas
complexities = Counter(ex.complexity.value for ex in all_examples)
print("\n📈 Distribución por complejidad:")
for comp, count in complexities.most_common():
    print(f"   {comp}: {count} ejemplos")

# Mostrar un ejemplo
print("\n📝 Ejemplo aleatorio:")
sample = random.choice(all_examples)
print(f"   Input: {sample.natural_language_input}")
print(f"   Fórmula: {sample.output.formula}")
print(f"   ASCII: {sample.output.formula_ascii}")